## CIFAR-10 with Neuron Shuffling Defense Mechanism

In this notebook, we perform on the CIFAR-10 dataset using the Resnet18 model and build models to defend against these attacks using the Neuron Shuffling mechanism.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import *

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import matplotlib.pyplot as plt
import numpy as np
import random
import itertools

from resnet import *

import pickle
from collections import OrderedDict

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.01

<a name='name'></a>
### Preparing train and test data and building Resnet model

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=30, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=20, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
net = ResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

Files already downloaded and verified
Files already downloaded and verified
==> Building model..


In [4]:
def train(epoch, net):
    
    '''
    this function train net on training dataset
    '''

    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        print(batch_idx)
    return train_loss/len(trainloader)

# Save the model
torch.save(net.state_dict(), 'resnet_model2.pth')

In [5]:
def test(epoch, net):

    '''
    This function evaluate net on test dataset
    '''

    global acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    test_accuracies = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = 100 * correct / total
    return test_loss/len(testloader)

In [ ]:
train_losses=[]
test_losses=[]
epochs=3

for epoch in range(0,epochs):
    train_losses.append(train(epoch, net))
    test_losses.append(test(epoch, net))
    scheduler.step()

## Defense Mechanism - Neuron Shuffling (with weight order)

In [6]:
# Function to shuffle layers and save models
def shuffle_and_save_layers(original_model_state_dict_path, shuffled_model_path1, shuffled_model_path2):
    # Load the state dictionary of the original model
    original_model_state_dict = torch.load(original_model_state_dict_path)
    
    # Assuming 'net' is defined somewhere in your code
    # Load the state dictionary into the model
    net.load_state_dict(original_model_state_dict)

    # Extract the layers from the model
    original_layers = list(net.children())

    # Shuffle the layers twice to get two different sets of shuffled layers
    shuffled_layers_1 = random.sample(original_layers, len(original_layers))
    shuffled_layers_2 = random.sample(original_layers, len(original_layers))

    # Concatenate original layers into a single model
    original_model = torch.nn.Sequential(*original_layers)

    torch.save(original_model, original.replace('.pth', '_model2.pth'))

    # Concatenate shuffled layers into a single model for both sets
    shuffled_model_1 = torch.nn.Sequential(*shuffled_layers_1)
    shuffled_model_2 = torch.nn.Sequential(*shuffled_layers_2)

    # Save the shuffled models and their weights
    torch.save(shuffled_model_1, shuffled_model_path1.replace('.pth', '_1.pth'))
    torch.save(shuffled_model_2, shuffled_model_path2.replace('.pth', '_2.pth'))

    # Store shuffled layers in an array
    shuffled_layers_array = [shuffled_layers_1, shuffled_layers_2]

    for i, (shuffled_layers, model_num) in enumerate(zip(shuffled_layers_array, [1, 2])):
        for j, shuffled_layer in enumerate(shuffled_layers):
            # Save entire models
            torch.save(shuffled_layer, f'shuffled_model_{model_num}_layer_{j}.pth')

            # Save model weights separately
            torch.save(shuffled_layer.state_dict(), f'shuffled_model_{model_num}_layer_{j}_weights.pth')

    return original_layers, shuffled_layers_array, original_model, [shuffled_model_1, shuffled_model_2]

# Example usage
original = 'resnet_model2.pth'
shuffled_model_path1 = 'shuffled_resnet.pth'
shuffled_model_path2 = 'shuffled_resnet.pth'

original_layers, shuffled_layers_array, original_model, shuffled_models = shuffle_and_save_layers(original, shuffled_model_path1, shuffled_model_path2)

# Print the original and shuffled layers along with their weights
for i, (original_layer, shuffled_layers) in enumerate(zip(original_layers, shuffled_layers_array)):
    print(f"Layer {i} - Original Weights:")
    print(original_layer.state_dict())
    for j, shuffled_layer in enumerate(shuffled_layers):
        print(f"\nLayer {i} - Shuffled Weights (Model {j+1}):")
        print(shuffled_layer.state_dict())
        print("\n" + "-"*50 + "\n")

# Print the original and shuffled models
print("Original Model Weights:")
print(original_model.state_dict())
print("\n" + "-"*50 + "\n")
for i, shuffled_model in enumerate(shuffled_models):
    print(f"Shuffled Model {i+1} Weights:")
    print(shuffled_model.state_dict())
    print("\n" + "-"*50 + "\n")

Layer 0 - Original Weights:
OrderedDict([('weight', tensor([[[[-0.0787,  0.1733,  0.1450],
          [ 0.1870,  0.1805, -0.0146],
          [ 0.1480, -0.0824, -0.0414]],

         [[-0.1338,  0.0958, -0.1734],
          [-0.0778,  0.0687,  0.1310],
          [-0.0687, -0.0558, -0.1156]],

         [[-0.1336, -0.1540,  0.1738],
          [ 0.1771,  0.1114,  0.0143],
          [-0.0692,  0.1264, -0.0771]]],


        [[[ 0.0311, -0.1418,  0.0042],
          [ 0.0207,  0.0077, -0.0680],
          [ 0.0034, -0.0906, -0.0958]],

         [[ 0.1593,  0.1643,  0.1803],
          [-0.1647,  0.0583, -0.1593],
          [-0.0124,  0.1389,  0.0946]],

         [[ 0.0023,  0.0409, -0.0074],
          [-0.0949, -0.1441, -0.0253],
          [ 0.0084, -0.1754, -0.0739]]],


        [[[ 0.0787, -0.0181,  0.1204],
          [-0.0476, -0.1209,  0.1201],
          [ 0.1836, -0.1358,  0.1712]],

         [[-0.0893, -0.1581,  0.1190],
          [-0.1661, -0.1538, -0.0505],
          [-0.0256,  0.1333, -0.0

### Testing Function

In [7]:
def test(epoch, net, shuffled_model_paths):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0

    # Test using shuffled layers
    use_original_layers = False
    layers_path = shuffled_model_paths[1] if use_original_layers else shuffled_model_paths[0]
    loaded_layers = torch.load(layers_path)

    if isinstance(loaded_layers, dict):
        loaded_state_dict = loaded_layers
    else:
        loaded_state_dict = loaded_layers.state_dict()

    for param_tensor in net.state_dict():
        if param_tensor in loaded_state_dict:
            net.state_dict()[param_tensor].copy_(loaded_state_dict[param_tensor])

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc_shuffled = 100 * correct / total

    return test_loss / len(testloader), acc_shuffled

In [8]:
def calculate_loss_using_acc(model, criterion, dataloader, accuracy):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            # Stop when the desired accuracy is reached
            if (correct / total) * 100 >= accuracy:
                break

    average_loss = total_loss / len(dataloader)
    return average_loss

In [10]:
train_losses = []
test_losses_shuffled = []
acc_shuffleds = []
epochs = 3

users = False  # Assuming this is a boolean indicating whether users are present or not
known_users = 1  # Number of known users
unknown_users = 2  # Number of unknown users

# Paths for saving shuffled models
shuffled_model_path1 = 'shuffled_resnet_1.pth'
shuffled_model_path2 = 'shuffled_resnet_2.pth'

for epoch in range(epochs):
    train_loss = train(epoch, net)  # Assuming you have a train function
    train_losses.append(train_loss)

    # Save shuffled models for each permutation
    for perm in itertools.permutations([1, 2]):
        shuffle_and_save_layers(original, shuffled_model_path1, shuffled_model_path2)

        # Test using shuffled layers
        test_loss_shuffled, acc_shuffled = test(epoch, net, [shuffled_model_path1, shuffled_model_path2])
        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"Train Loss: {train_loss}")
        print(f"Test Loss using Shuffled Layers: {test_loss_shuffled}")
        test_losses_shuffled.append(test_loss_shuffled)
        acc_shuffleds.append(acc_shuffled)

    scheduler.step()

if users and known_users >= 1:
    print('Original Accuracy : %d %%' % (acc))

if not users and unknown_users >= 1:
    print('Wrong Accuracies: ', acc_shuffleds)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27